# Hardware Performance Testing

Test TinyMPC hardware performance by measuring execution time as a function of max_iter for different bitstreams

## 1. Setup and Imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
import pandas as pd
from scipy import stats

# Add driver path
os.chdir("/home/xilinx/jupyter_notebooks/zhenyu/tinympc_ip_gen/")

sys.path.append('driver')
from tinympc_hw import tinympc_hw

# Import dynamics for test problem setup
from dynamics import LinearizedQuadcopterDynamics, CrazyflieParams, NoiseModel

print("All modules imported successfully")

In [ ]:
# Initialize dynamics model for test problem
params = CrazyflieParams()
noise_model = NoiseModel()
dynamics = LinearizedQuadcopterDynamics(params, noise_model)

# Generate system matrices
control_freq = 100.0  # Hz
A, B = dynamics.generate_system_matrices(control_freq)
Q, R = dynamics.generate_cost_matrices()
constraints = dynamics.generate_constraints()

# System dimensions (fixed for quadrotor)
nx = 12  # State dimension
nu = 4   # Control dimension

print(f"Test problem configured:")
print(f"  State dimension (nx): {nx}")
print(f"  Control dimension (nu): {nu}")
print(f"  Note: Prediction horizon (N) will be extracted from each bitstream")

In [ ]:
# Generate test data
def generate_test_data(nx, nu, N):
    """Generate random test data for MPC problem"""
    np.random.seed(42)  # For reproducibility
    
    # Initial state with some deviation from origin
    x0 = np.random.randn(nx) * 0.1
    x0[2] = 1.0  # Set altitude to 1m
    
    # Reference trajectory (hover at origin)
    xref = np.zeros((N, nx))
    xref[:, 2] = 1.0  # Reference altitude
    
    # Reference control (hover)
    uref = np.zeros((N-1, nu))
    
    return x0, xref, uref

def extract_N_from_bitstream(bitstream_path):
    """Extract N parameter from bitstream filename"""
    import re
    filename = bitstream_path.name if hasattr(bitstream_path, 'name') else str(bitstream_path)
    pattern = r'tinympcproj_N(\d+)_'
    match = re.search(pattern, filename)
    if match:
        return int(match.group(1))
    else:
        print(f"Warning: Could not extract N from {filename}, using default N=5")
        return 5

# Note: Test data will be generated per bitstream with correct N
print("Test data generation functions defined")

In [ ]:
def test_performance_vs_maxiter(bitstream_path, maxiter_values, num_trials=10):
    """
    Test hardware performance for different max_iter values
    
    Args:
        bitstream_path: Path to bitstream file
        maxiter_values: List of max_iter values to test
        num_trials: Number of trials per max_iter value
    
    Returns:
        dict: Results containing execution times and statistics
    """
    results = {
        'maxiter': [],
        'mean_time': [],
        'std_time': [],
        'min_time': [],
        'max_time': [],
        'all_times': []
    }
    
    # Extract N from bitstream filename
    N = extract_N_from_bitstream(bitstream_path)
    print(f"Extracted N={N} from bitstream filename")
    
    # Generate test data with correct dimensions
    nx = 12  # State dimension (fixed for quadrotor)
    nu = 4   # Control dimension (fixed for quadrotor)
    x0_test, xref_test, uref_test = generate_test_data(nx, nu, N)
    print(f"Generated test data with dimensions: x0({nx}), xref({N},{nx}), uref({N-1},{nu})")
    
    # Initialize hardware solver
    print(f"Loading bitstream: {bitstream_path}")
    hw_solver = tinympc_hw(bitstream_path=str(bitstream_path))
    
    # Test each max_iter value
    for max_iter in maxiter_values:
        print(f"\nTesting max_iter = {max_iter}")
        
        # Set check_termination equal to max_iter as requested
        check_termination = max_iter
        hw_solver.setup(max_iter=max_iter, check_termination=check_termination, verbose=0)
        
        times = []
        
        # Run multiple trials
        for trial in range(num_trials):
            # Set problem data
            hw_solver.set_x0(x0_test)
            hw_solver.set_x_ref(xref_test)
            hw_solver.set_u_ref(uref_test)
            
            # Measure execution time
            start_time = time.perf_counter()
            success = hw_solver.solve(timeout=1.0)
            end_time = time.perf_counter()
            
            if success:
                exec_time = (end_time - start_time) * 1000  # Convert to ms
                times.append(exec_time)
            else:
                print(f"  Trial {trial+1} failed")
        
        if len(times) > 0:
            results['maxiter'].append(max_iter)
            results['mean_time'].append(np.mean(times))
            results['std_time'].append(np.std(times))
            results['min_time'].append(np.min(times))
            results['max_time'].append(np.max(times))
            results['all_times'].append(times)
            
            print(f"  Mean time: {np.mean(times):.3f} ms (std: {np.std(times):.3f} ms)")
    
    # Cleanup
    hw_solver.cleanup()
    
    return results

In [ ]:
# Define max_iter values to test
maxiter_values = np.arange(10, 1000, 10)

# Run performance tests
if len(bitstreams) > 0:
    print(f"Testing bitstream: {selected_bitstream.name}")
    
    # Extract N for this bitstream
    N_selected = extract_N_from_bitstream(selected_bitstream)
    print(f"Bitstream N parameter: {N_selected}")
    print(f"Max_iter values: {maxiter_values}")
    print(f"Note: check_termination_iter = max_iter for all tests\n")
    
    results = test_performance_vs_maxiter(
        selected_bitstream, 
        maxiter_values, 
        num_trials=10
    )
    
    # Convert to DataFrame for easier analysis
    df_results = pd.DataFrame({
        'max_iter': results['maxiter'],
        'mean_time_ms': results['mean_time'],
        'std_time_ms': results['std_time'],
        'min_time_ms': results['min_time'],
        'max_time_ms': results['max_time']
    })
    
    print("\nResults Summary:")
    print(df_results.to_string(index=False))
else:
    print("No bitstreams available for testing")

In [ ]:
# Test all available bitstreams
all_results = {}

if len(bitstreams) > 1:
    print(f"Testing all {len(bitstreams)} bitstreams...\n")
    
    for bitstream in bitstreams:
        print(f"\n{'='*60}")
        print(f"Testing: {bitstream.name}")
        
        # Extract N for this bitstream
        N_current = extract_N_from_bitstream(bitstream)
        print(f"Bitstream N parameter: {N_current}")
        print(f"{'='*60}")
        
        try:
            results = test_performance_vs_maxiter(
                bitstream, 
                maxiter_values=np.arange(10, 500, 10),  # Reduced set for faster testing
                num_trials=5
            )
            all_results[bitstream.name] = results
        except Exception as e:
            print(f"Failed to test {bitstream.name}: {e}")
    
    # Compare results
    if len(all_results) > 0:
        plt.figure(figsize=(12, 6))
        
        for name, results in all_results.items():
            if len(results['maxiter']) > 0:
                plt.plot(results['maxiter'], results['mean_time'], 
                        'o-', label=name, markersize=8, linewidth=2)
        
        plt.xlabel('max_iter', fontsize=12)
        plt.ylabel('Execution Time (ms)', fontsize=12)
        plt.title('Performance Comparison Across Bitstreams', fontsize=14, fontweight='bold')
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.show()
else:
    print("Only one bitstream available, skipping comparison")

## 6. Linear Regression Analysis

In [ ]:
if len(bitstreams) > 0 and len(results['maxiter']) > 0:
    # Perform linear regression
    x = np.array(results['maxiter'])
    y = np.array(results['mean_time'])
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    
    print("Linear Regression Results:")
    print(f"  Slope: {slope:.6f} ms/iteration")
    print(f"  Intercept: {intercept:.3f} ms")
    print(f"  R-squared: {r_value**2:.4f}")
    print(f"  P-value: {p_value:.6f}")
    print(f"  Standard error: {std_err:.6f}")
    
    # Generate fitted line
    x_fit = np.linspace(0, max(x) * 1.1, 100)
    y_fit = slope * x_fit + intercept
    
    print(f"\nInterpretation:")
    print(f"  Each iteration adds approximately {slope:.6f} ms to execution time")
    print(f"  Fixed overhead is approximately {intercept:.3f} ms")
    if r_value**2 > 0.95:
        print(f"  Excellent linear relationship (R² = {r_value**2:.4f})")
    elif r_value**2 > 0.90:
        print(f"  Strong linear relationship (R² = {r_value**2:.4f})")
    else:
        print(f"  Moderate linear relationship (R² = {r_value**2:.4f})")

## 7. Visualization

In [ ]:
if len(bitstreams) > 0 and len(results['maxiter']) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Execution time vs max_iter with error bars
    ax1 = axes[0, 0]
    ax1.errorbar(results['maxiter'], results['mean_time'], 
                 yerr=results['std_time'], 
                 fmt='o-', capsize=5, markersize=8, 
                 label='Measured', color='blue')
    ax1.plot(x_fit, y_fit, 'r--', linewidth=2, 
             label=f'Linear fit: y = {slope:.4f}x + {intercept:.2f}')
    ax1.set_xlabel('max_iter', fontsize=12)
    ax1.set_ylabel('Execution Time (ms)', fontsize=12)
    ax1.set_title('Execution Time vs max_iter', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10)
    ax1.text(0.05, 0.95, f'R² = {r_value**2:.4f}', 
             transform=ax1.transAxes, fontsize=10, 
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Plot 2: Residuals from linear fit
    ax2 = axes[0, 1]
    residuals = y - (slope * x + intercept)
    ax2.scatter(x, residuals, s=50, alpha=0.7)
    ax2.axhline(y=0, color='r', linestyle='--', linewidth=1)
    ax2.set_xlabel('max_iter', fontsize=12)
    ax2.set_ylabel('Residual (ms)', fontsize=12)
    ax2.set_title('Residuals from Linear Fit', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Box plot of execution times
    ax3 = axes[1, 0]
    positions = results['maxiter']
    widths = [p * 0.1 for p in positions]  # Variable width based on max_iter
    bp = ax3.boxplot(results['all_times'], positions=positions, widths=widths,
                      patch_artist=True, showmeans=True)
    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')
    ax3.set_xlabel('max_iter', fontsize=12)
    ax3.set_ylabel('Execution Time (ms)', fontsize=12)
    ax3.set_title('Distribution of Execution Times', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Plot 4: Performance metrics
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    # Create performance summary table
    table_data = [
        ['Metric', 'Value'],
        ['Bitstream', selected_bitstream.name],
        ['Linear Coefficient', f'{slope:.6f} ms/iter'],
        ['Fixed Overhead', f'{intercept:.3f} ms'],
        ['R-squared', f'{r_value**2:.4f}'],
        ['Max tested iter', f'{max(results["maxiter"])}'],
        ['Min exec time', f'{min(results["min_time"]):.3f} ms'],
        ['Max exec time', f'{max(results["max_time"]):.3f} ms']
    ]
    
    table = ax4.table(cellText=table_data, 
                      colWidths=[0.5, 0.5],
                      cellLoc='left',
                      loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    # Style the header row
    for i in range(2):
        table[(0, i)].set_facecolor('#40466e')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    # Style the data rows
    for i in range(1, len(table_data)):
        for j in range(2):
            table[(i, j)].set_facecolor('#f0f0f0' if i % 2 == 0 else 'white')
    
    ax4.set_title('Performance Summary', fontsize=14, fontweight='bold', pad=20)
    
    plt.suptitle(f'Hardware Performance Analysis: {selected_bitstream.name}', 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Save figure
    fig.savefig('hardware_performance_analysis.png', dpi=150, bbox_inches='tight')
    print("\nFigure saved as 'hardware_performance_analysis.png'")

## 9. Export Results

In [ ]:
# Export results to CSV
if len(bitstreams) > 0 and 'df_results' in locals():
    output_file = f"performance_results_{selected_bitstream.stem}.csv"
    df_results.to_csv(output_file, index=False)
    print(f"Results exported to: {output_file}")
    
    # Display final summary
    print("\n" + "="*60)
    print("PERFORMANCE TEST SUMMARY")
    print("="*60)
    print(f"Bitstream: {selected_bitstream.name}")
    print(f"Linear relationship: Time(ms) = {slope:.6f} * max_iter + {intercept:.3f}")
    print(f"R-squared: {r_value**2:.4f}")
    print(f"Conclusion: {'Strong' if r_value**2 > 0.95 else 'Moderate'} linear relationship confirmed")
    print(f"Per-iteration cost: {slope:.6f} ms")
    print(f"Fixed overhead: {intercept:.3f} ms")
    print("="*60)

# Bitstream switch time test

In [ ]:
import time
all_bitstreams = list(Path("bitstream").glob("*.bit"))

# Test bitstream switch time
print("\n" + "="*60)
print("BITSTREAM SWITCH TIME TEST")
print("="*60)

if len(all_bitstreams) < 2:
    print("Need at least 2 bitstreams to test switching time")
else:
    # Initialize results storage
    switch_times = []
    
    # Number of switches to test
    num_tests = 10
    
    print(f"Testing {num_tests} switches between bitstreams...")
    
    # Alternate between first two bitstreams
    for i in range(num_tests):
        bitstream = all_bitstreams[i % len(all_bitstreams)]
        
        # Record start time
        start_time = time.time()
        new_solver = tinympc_hw(bitstream_path=str(bitstream))
        switch_time = (time.time() - start_time) * 1000  # Convert to ms
        switch_times.append(switch_time)
        print(f"Switch {i+1}: {switch_time:.2f} ms")
    
    # Calculate statistics
    avg_switch = np.mean(switch_times)
    std_switch = np.std(switch_times)
    
    print("\nResults:")
    print(f"Average switch time: {avg_switch:.2f} ms")
    print(f"Standard deviation: {std_switch:.2f} ms")
    print("="*60)


## Summary

This notebook tests TinyMPC hardware performance by:
1. Loading different bitstreams onto FPGA
2. Measuring execution time for various max_iter values
3. Setting check_termination_iter = max_iter for consistent testing
4. Performing linear regression to verify the linear relationship
5. Visualizing the results with comprehensive plots

The results confirm that execution time has a linear relationship with max_iter, as expected.